In [25]:
import torch
from torch.utils.data import DataLoader, Dataset
import numpy as np
import random, math
from tqdm import tqdm

# Define a simple custom dataset
class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = {
            'data': self.data[idx],
            'label': self.labels[idx]
        }
        return sample




In [26]:
def train_val_split(dataset, prop = 0.9):
    authors = list(set(dataset.labels))
    random.shuffle(authors)

    train_len = math.floor(len(authors) * prop)
    train_authors = authors[:train_len]
    val_authors = authors[train_len:]

    print(f'{len(train_authors)} authors for training - {len(val_authors)} authors for validation')

    train_idxs = []
    val_idxs = []

    for i in tqdm(range(len(dataset)), desc='Splitting dataset'):
        w = dataset.get_label(i)[1]
        if w in train_authors:
            train_idxs.append(i)
        if w in val_authors:
            val_idxs.append(i)

    train = torch.utils.data.Subset(dataset, train_idxs)
    val = torch.utils.data.Subset(dataset, val_idxs)

    return train, val

In [27]:
# Generate some random data
data = np.random.randn(100, 3)  # 100 samples, each with 3 features
labels = np.random.randint(0, 2, 100)  # Binary labels for each sample

breakpoint()

# Create an instance of MyDataset
dataset = MyDataset(data, labels)




In [28]:
dataset.labels

array([0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1])

In [29]:
train_ds, val_ds = train_val_split(dataset)

1 authors for training - 1 authors for validation


Splitting dataset:   0%|                                                                                                                                                            | 0/100 [00:00<?, ?it/s]


AttributeError: 'MyDataset' object has no attribute 'get_label'

In [ ]:
# Create a DataLoader
batch_size = 10
breakpoint()
dataloader = DataLoader(train_ds, num_workers=4, batch_size=batch_size, shuffle=True)

# Iterate through the DataLoader
for batch_idx, batch in enumerate(dataloader):
    print(f"Batch {batch_idx + 1}")
    print("Data:", batch['data'])
    print("Labels:", batch['label'])
    print()  # Blank line for readability

In [2]:
import torch

from Smooth_AP_loss import SmoothAP
from Smooth_AP_loss_o import SmoothAP as SmoothAP1
from Smooth_AP_loss2 import SmoothAP as SmoothAP2
from Smooth_AP_loss3 import SoftAPLoss as SmoothAP3


In [37]:
torch.manual_seed(42)  # For reproducibility
batch_size = 16*4
num_classes = 4
feat_dims = 5
anneal = 0.01
labels = torch.tensor([i // int(batch_size/num_classes) for i in range(batch_size)])

In [38]:
inputs = torch.randn(batch_size, feat_dims, requires_grad=True)

In [13]:
SmoothAP(anneal=anneal, batch_size=batch_size, num_id=num_classes, feat_dims=feat_dims)
SmoothAP1(anneal=anneal, batch_size=batch_size, num_id=num_classes, feat_dims=feat_dims)
SmoothAP2(anneal=anneal, batch_size=batch_size, num_id=num_classes, feat_dims=feat_dims)
smooth_ap_loss3 = SmoothAP3(anneal=anneal)


In [14]:
loss_initial = smooth_ap_loss(inputs)
loss_initial

tensor([0.6556], grad_fn=<RsubBackward1>)

In [16]:
loss_initial1 = smooth_ap_loss1(inputs)
loss_initial1

tensor([0.6556], grad_fn=<RsubBackward1>)

In [17]:
loss_initial2 = smooth_ap_loss2(inputs)
loss_initial2

tensor([0.6556], grad_fn=<RsubBackward1>)

In [19]:
loss_initial3 = smooth_ap_loss3(inputs, labels)
loss_initial3

tensor(0.7053, grad_fn=<RsubBackward1>)

In [10]:
labels = torch.tensor([i // int(batch_size/num_classes) for i in range(batch_size)])
labels

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [26]:
def measure_time_and_memory(loss_fn, embeddings):
    torch.cuda.reset_peak_memory_stats()
    start_time = time.time()
    loss = loss_fn(embeddings)
    loss.backward()
    end_time = time.time()
    memory_allocated = torch.cuda.max_memory_allocated() / 1024**2  # Convert to MB
    elapsed_time = end_time - start_time
    return elapsed_time, memory_allocated

In [27]:
loss_fns = [
        SmoothAP(anneal=anneal, batch_size=batch_size, num_id=num_classes, feat_dims=feat_dims),
        SmoothAP1(anneal=anneal, batch_size=batch_size, num_id=num_classes, feat_dims=feat_dims),
        SmoothAP2(anneal=anneal, batch_size=batch_size, num_id=num_classes, feat_dims=feat_dims)
    ]

for i, loss_fn in enumerate(loss_fns, start=1):
    elapsed_time, memory_allocated = measure_time_and_memory(loss_fn, inputs)
    print(f"SmoothAPLoss{i}: Time = {elapsed_time:.4f}s, Memory = {memory_allocated:.2f}MB")


RuntimeError: invalid argument to reset_peak_memory_stats

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [30]:
inputs = torch.randn(batch_size, feat_dims, requires_grad=True).to(device)

In [32]:
inputs.shape()

TypeError: 'torch.Size' object is not callable

In [33]:
import torch

In [41]:
torch.manual_seed(42)  # For reproducibility
batch_size = 4*4
num_classes = 4
feat_dims = 5
anneal = 0.01
labels = torch.tensor([i // int(batch_size/num_classes) for i in range(batch_size)])

In [42]:
inputs = torch.randn(batch_size, feat_dims, requires_grad=True)

In [43]:
inputs

tensor([[ 1.9269,  1.4873,  0.9007, -2.1055,  0.6784],
        [-1.2345, -0.0431, -1.6047, -0.7521,  1.6487],
        [-0.3925, -1.4036, -0.7279, -0.5594, -0.7688],
        [ 0.7624,  1.6423, -0.1596, -0.4974,  0.4396],
        [-0.7581,  1.0783,  0.8008,  1.6806,  1.2791],
        [ 1.2964,  0.6105,  1.3347, -0.2316,  0.0418],
        [-0.2516,  0.8599, -1.3847, -0.8712, -0.2234],
        [ 1.7174,  0.3189, -0.4245,  0.3057, -0.7746],
        [-1.5576,  0.9956, -0.8798, -0.6011, -1.2742],
        [ 2.1228, -1.2347, -0.4879, -0.9138, -0.6581],
        [ 0.0780,  0.5258, -0.4880,  1.1914, -0.8140],
        [-0.7360, -1.4032,  0.0360, -0.0635,  0.6756],
        [-0.0978,  1.8446, -1.1845,  1.3835,  1.4451],
        [ 0.8564,  2.2181,  0.5232,  0.3466, -0.1973],
        [-1.0546,  1.2780, -0.1722,  0.5238,  0.0566],
        [ 0.4263,  0.5750, -0.6417, -2.2064, -0.7508]], requires_grad=True)

In [46]:
distance = torch.matmul(inputs, inputs.t())

In [47]:
distance.shape

torch.Size([16, 16])

In [48]:
dis = torch.mm(inputs, inputs.t())

In [50]:
dis.shape

torch.Size([16, 16])

In [52]:
torch.equal(distance,dis)

True

In [53]:
n = torch.randint(1,3,(4,4))

In [54]:
n

tensor([[1, 2, 1, 2],
        [1, 1, 2, 1],
        [2, 2, 2, 2],
        [2, 2, 2, 2]])

In [55]:
n - n[:,0]

tensor([[ 0,  1, -1,  0],
        [ 0,  0,  0, -1],
        [ 1,  1,  0,  0],
        [ 1,  1,  0,  0]])

In [56]:
n - n[:,0:1]

tensor([[0, 1, 0, 1],
        [0, 0, 1, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]])